In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from datetime import datetime,date,timedelta
import pickle

In [8]:
allwind = pd.read_pickle('brommer_wind_2020-10-04_2022-10-03.pkl')

In [9]:
now = datetime.now().date()
sdate= datetime.now().date()-timedelta(days=365*2)
datediff = (now-sdate).days

In [10]:
pd.concat([allwind[:3],allwind[-3:]]) 

,Date,Time,Temp,Wind,Orient,Speed,Gust
0,2020-10-04,12:04 AM,55.6,East,90.0,0.0,0.0
1,2020-10-04,12:09 AM,55.4,East,90.0,0.0,0.0
2,2020-10-04,12:14 AM,55.3,East,90.0,0.0,0.0
285,2022-10-03,11:49 PM,57.6,West,270.0,0.0,2.0
286,2022-10-03,11:54 PM,57.4,West,270.0,0.0,2.0
287,2022-10-03,11:59 PM,57.4,West,270.0,0.0,2.0


In [184]:
dfcov=pd.DataFrame(columns=['Date','Records','PctTotal','NoDir','PctNoDir'])
fullcov=0
for i in range(365*2):
    d = sdate+timedelta(days=i)
    itvls = len(allwind[allwind['Date']==d])
    cov = round(100*itvls/(12*24),1)
    nullwcov = (allwind[allwind['Date']==d]['Wind']=='').sum()
    if cov == 100.0:
        fullcov+=1
    if itvls != 0:
        pctnullw = round(100*nullwcov/itvls,1)
    else:
        pctnullw = 0.0
    rdata = pd.DataFrame([{'Date':d,'Records':itvls,'PctTotal':cov,'NoDir':nullwcov,'PctNoDir':pctnullw}])
    dfcov = pd.concat([dfcov,rdata])

In [98]:
print('duplicate entries on a couple dates:')
dfcov.sort_values(['PctTotal','PctNoDir'],ascending=[False,True]).head(3)

duplicate entries on a couple dates:


,Date,Records,PctTotal,NoDir,PctNoDir
0,2020-11-01,300,104.2,0,0.0
0,2021-11-07,300,104.2,41,13.7
0,2020-10-04,288,100.0,0,0.0


In [183]:
from collections import Counter
cdict = Counter(list(allwind[allwind['Date']==date(2020,11,1)].Time))
print("2020-11-01 extras:")
print({x:count for x,count in cdict.items() if count > 1})
cdict2 = Counter(list(allwind[allwind['Date']==date(2021,11,7)].Time))
print("\n2021-11-07 extras:")
print({x:count for x,count in cdict2.items() if count > 1})

2020-11-01 extras:
{'1:04 AM': 2, '1:09 AM': 2, '1:14 AM': 2, '1:19 AM': 2, '1:24 AM': 2, '1:29 AM': 2, '1:34 AM': 2, '1:39 AM': 2, '1:44 AM': 2, '1:49 AM': 2, '1:54 AM': 2, '1:59 AM': 2}

2021-11-07 extras:
{'1:04 AM': 2, '1:09 AM': 2, '1:14 AM': 2, '1:19 AM': 2, '1:24 AM': 2, '1:29 AM': 2, '1:34 AM': 2, '1:39 AM': 2, '1:44 AM': 2, '1:49 AM': 2, '1:54 AM': 2, '1:59 AM': 2}


In [163]:
print('Entries from 1AM-2AM are recorded twice, with different readings oddly enough')
allwind[(allwind['Date']==date(2020,11,1)) & (allwind['Time'].str[:3] == '1:0') & (allwind['Time'].str[-2] == 'A')]

Entries from 1AM-2AM are recorded twice, with different readings oddly enough


,Date,Time,Temp,Wind,Orient,Speed,Gust
12,2020-11-01,1:04 AM,53.1,East,90.0,0.0,0.0
13,2020-11-01,1:09 AM,52.9,East,90.0,0.0,0.0
24,2020-11-01,1:04 AM,51.6,East,90.0,0.0,0.0
25,2020-11-01,1:09 AM,51.4,East,90.0,0.0,0.0


In [161]:
#using 'startswith' instead of indexing
allwind[(allwind['Date']==date(2021,11,7)) & (allwind['Time'].str.startswith('1:0'))& (allwind['Time'].str.endswith('AM')) ]

,Date,Time,Temp,Wind,Orient,Speed,Gust
12,2021-11-07,1:04 AM,56.3,NNW,337.5,3.8,8.0
13,2021-11-07,1:09 AM,56.2,NNW,337.5,3.1,7.1
24,2021-11-07,1:04 AM,54.9,NNW,337.5,4.2,8.0
25,2021-11-07,1:09 AM,55.0,NNW,337.5,4.1,8.0


In [186]:
print(f"# days with full coverage of all 5-min intervals: {fullcov} ({round(100*(fullcov/datediff),1)}%)")

# days with full coverage of all 5-min intervals: 497 (68.1%)


In [187]:
print("large gap in coverage:")
pd.concat([allwind[allwind['Date']==date(2022, 9, 21)][-2:],allwind[allwind['Date']==date(2022, 9, 22)][:2]]) 

large gap in coverage:


,Date,Time,Temp,Wind,Orient,Speed,Gust
157,2022-09-21,3:14 PM,71.9,SW,225.0,4.8,12.0
158,2022-09-21,3:19 PM,71.5,SSW,202.5,5.3,11.1
0,2022-09-22,8:24 AM,58.1,NNE,22.5,0.7,2.0
1,2022-09-22,8:29 AM,58.7,NNE,22.5,0.7,2.0


In [ ]:
dfcov.to_pickle(f'brommer_wind_coverage_{sdate}_{now-timedelta(days=1)}.pkl')

In [190]:
dfcov.to_csv(f'brommer_wind_coverage_{sdate}_{now-timedelta(days=1)}.csv')